In [1]:
import yapo
import numpy as np

## >>> Инициализация данных. Не будет в конечной статье

In [2]:
snp_asset = yapo.portfolio_asset(name='ny/SPY', 
                                 start_period='2015-1', end_period='2016-3', currency='usd')
print(snp_asset.close())
snp_values = snp_asset.close().values
snp_values

TimeSeries(start_period=2015-01, end_period=2016-03, kind=TimeSeriesKind.VALUES, values=[183.30505169 193.60763852 190.55948884 192.43343373 194.90738578
 190.95253351 195.26600501 183.36451696 178.70115422 193.90141823
 194.61015112 191.26732302 181.74477321 181.59466115 193.80591971]


array([183.30505169, 193.60763852, 190.55948884, 192.43343373,
       194.90738578, 190.95253351, 195.26600501, 183.36451696,
       178.70115422, 193.90141823, 194.61015112, 191.26732302,
       181.74477321, 181.59466115, 193.80591971])

In [3]:
infl_usd = yapo.inflation(currency='usd', kind='values', start_period='2015-1', end_period='2016-3')
print(infl_usd)
infl_usd_values = infl_usd.values
infl_usd_values

TimeSeries(start_period=2015-01, end_period=2016-03, kind=TimeSeriesKind.DIFF, values=[-4.706e-03  4.343e-03  5.952e-03  2.033e-03  5.097e-03  3.503e-03
  6.700e-05 -1.416e-03 -1.557e-03 -4.500e-04 -2.111e-03 -3.417e-03
  1.653e-03  8.230e-04  4.306e-03]


array([-4.706e-03,  4.343e-03,  5.952e-03,  2.033e-03,  5.097e-03,
        3.503e-03,  6.700e-05, -1.416e-03, -1.557e-03, -4.500e-04,
       -2.111e-03, -3.417e-03,  1.653e-03,  8.230e-04,  4.306e-03])

## ??? ЗАГОЛОВОК СТАТЬИ ???

Ошибки при написании программ неизбежны. В этой статье будет рассказан один из способов, которым можно контролировать правильность выполнения операций над временными рядами.

Начнём с создания данных для дввх временных рядов: 
- значения индекса S&P с января 2015 года по март 2016
- значения инфляции США с января 2015 года по март 2016

In [4]:
snp_values

array([183.30505169, 193.60763852, 190.55948884, 192.43343373,
       194.90738578, 190.95253351, 195.26600501, 183.36451696,
       178.70115422, 193.90141823, 194.61015112, 191.26732302,
       181.74477321, 181.59466115, 193.80591971])

In [5]:
infl_usd_values

array([-4.706e-03,  4.343e-03,  5.952e-03,  2.033e-03,  5.097e-03,
        3.503e-03,  6.700e-05, -1.416e-03, -1.557e-03, -4.500e-04,
       -2.111e-03, -3.417e-03,  1.653e-03,  8.230e-04,  4.306e-03])

Посчитаем реальное значение индекса S&P:

In [6]:
(snp_values + 1.) / (infl_usd_values + 1.) - 1.

array([184.17649226, 192.76611229, 189.42607285, 192.04098141,
       193.91390959, 190.28247101, 195.25285607, 183.6259473 ,
       178.98138524, 193.98916336, 195.02395769, 191.92655305,
       181.44319461, 181.44450932, 192.97068195])

Несмотря на успешность выполнения, в этом подсчёте нет никакого смысла: реальные значения считаются от относительных изменений индекса, а не от его абсолютных значений. Ни `numpy.array`, ни встроенные средства `Python` не подсказывают об ошибке.

Как говорилось в [предыдущей статье](https://rostsber.ru/publish/stocks/python_asset.html), накопленная доходность считается так:

In [7]:
snp_ror = np.diff(snp_values) / snp_values[:-1]
# check:
# print(snp_asset.rate_of_return().values - snp_ror)
snp_ror

array([ 0.0562046 , -0.01574395,  0.00983391,  0.01285614, -0.02029093,
        0.02258923, -0.06095013, -0.0254322 ,  0.08505969,  0.00365512,
       -0.01717705, -0.0497866 , -0.00082595,  0.06724459])

Посчитаем **реальные значения** накопленной доходности. Какой из следующих двух вариантов подсчёта реальной доходности правильный?

In [8]:
snp_ror_real_1 = (snp_ror + 1.) / (infl_usd_values[1:] + 1.) - 1.
snp_ror_real_1

array([ 0.05163734, -0.02156758,  0.00778508,  0.0077198 , -0.02371087,
        0.02252073, -0.05961855, -0.02391243,  0.08554818,  0.00577832,
       -0.01380723, -0.05135471, -0.00164759,  0.06266874])

In [9]:
snp_ror_real_2 = (snp_ror + 1.) / (infl_usd_values[:-1] + 1.) - 1.
snp_ror_real_2

array([ 0.0611986 , -0.02000009,  0.00385894,  0.01080119, -0.02525918,
        0.01901961, -0.06101304, -0.02405025,  0.08675176,  0.00410697,
       -0.01509792, -0.04652859, -0.00247486,  0.06636697])

In [10]:
# check
snp_asset.rate_of_return(real=True).values - snp_ror_real_1

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

Правильный ответ: первый. Проблема была в неправильной проекции `infl_usd_values`: во втором варианте временной ряд доходности с февраля 2015 по март 2016 была поделена на временной ряд инфляции с января 2015 по февраль 2016 года. Такие ошибки возможно детектировать во время исполнения программы.

Теперь посчитаем реальное значение CAGR за 1 год:

In [11]:
years_ago = 1
snp_cagr = (snp_ror[-years_ago * 12:] + 1.).prod() ** (1 / years_ago) - 1.
# check:
print(snp_asset.compound_annual_growth_rate(years_ago=1).value - snp_cagr)
snp_cagr

0.0


0.017036311812289373

In [12]:
infl_usd_values[-years_ago*12:]

array([ 2.033e-03,  5.097e-03,  3.503e-03,  6.700e-05, -1.416e-03,
       -1.557e-03, -4.500e-04, -2.111e-03, -3.417e-03,  1.653e-03,
        8.230e-04,  4.306e-03])

In [13]:
# check
print(snp_ror[-years_ago * 12:].shape)
print(infl_usd_values[-years_ago * 12:].shape)

(12,)
(12,)


In [14]:
infl_usd_accumulated = (infl_usd_values[-years_ago * 12 + 1:] + 1.).prod() - 1.
snp_cagr_real = (snp_cagr + 1.) / (infl_usd_accumulated + 1.) - 1.
snp_cagr_real

0.010489875626582323

В канве статьи, здесь также есть ошибка. Оказывается, по аналогии с расчётом `snp_ror_real` программист решил взять инфляцию без первого значения, т.е. за 11 месяцев вместо 12, затем посчитал аккумулированное значение, а по сути разделил одно число, равное произведению 12 чисел, на другое число, равное произведению 11 чисел!

Правильный расчёт будет такой:

In [15]:
years_ago = 1
snp_cagr = (snp_ror[-years_ago * 12:] + 1.).prod() ** (1 / years_ago) - 1.
# check:
print(snp_asset.compound_annual_growth_rate(years_ago=1).value)
print(snp_cagr)

infl_usd_accumulated = (infl_usd_values[-years_ago * 12:] + 1.).prod() - 1.
snp_cagr_real = (snp_cagr + 1.) / (infl_usd_accumulated + 1.) - 1.
# check:
print(snp_asset.compound_annual_growth_rate(years_ago=1, real=True).value)
snp_cagr_real

0.017036311812289373
0.017036311812289373
0.00843971768053775


0.00843971768053775

Для отлова таких ошибок необходима "расширенная" версия `np.array`, которая бы:

- имела мета-информацию над значениями `np.array`: начало и конец периода, уровень частичных значений (чтобы отличать значения от прироста, и "прироста от прироста" более высоких подярков, если такое понадобиться)
    - например, уровень частичных значений для индекса S&P равен 0, а для инфляции и накопленной доходности -- 1 
- иметь такой же интерфейс методов как `np.array`, включая арифметические операции, достаточное для текущих задач множество, но расширяемое по мере необходимости
- валидировать каждый вызов метода: соответствующие начальный период, конечный период, и уровень частичных значений должны совпадать

Мне известны два принципиальных способа:
- вести реестр (хеш-таблица), в которой записывать всю мета-информацию, но всё равно придётся перегружать стандартные алгебраические операции, чтобы провалидировать данные
- расширить класс `np.array` через композицию или наследование. Расширение через наследование, согласно [numpy API](https://docs.scipy.org/doc/numpy-1.15.0/user/basics.subclassing.html), особых преимуществ не даёт, а напротив может создать проблемы в случае изменения `numpy API`. Поэтому будет расширять через композицию:

In [16]:
import pandas as pd

class TimeSeries:
    def __init__(self, values, start_period: pd.Period, end_period: pd.Period, diff_level):
        if not isinstance(values, np.ndarray):
            raise ValueError('values should be numpy array')
        if len(values) != end_period - start_period + 1:
            raise ValueError('values and period range has different lengths')
        self.values = values
        self.start_period = start_period
        self.end_period = end_period
        self.diff_level = diff_level
    
    def __validate(self, time_series):
        if self.start_period != time_series.start_period:
            raise ValueError('start periods are incompatible')
        if self.end_period != time_series.end_period:
            raise ValueError('end periods are incompatible')
        if self.diff_level != time_series.diff_level:
            raise ValueError('diff levels are incompatible')
    
    def apply(self, fun, *args):
        '''
        Обобщённый метод для применения произвольной функции `fun` с аргументами `args` 
        к текущему экземпляру `TimeSeries`
        '''
        
        # Сейчас TimeSeries поддерживает функции с 0 и 1 аргументом
        
        # Пример функции без аргументов: np.array([2, 4]).cumprod() ~> np.array([2, 8])
        if len(args) == 0:
            ts = TimeSeries(values=fun(self.values),
                            start_period=self.start_period, end_period=self.end_period,
                            diff_level=self.diff_level)
            return ts
        
        # Сейчас TimeSeries в качестве второго аргумента поддерживает только TimeSeries или скаляр
        else:
            other = args[0]
            if isinstance(other, TimeSeries):
                self.__validate(other) # проверим, что TimeSeries совместимы
                # для совместимых просто посчитаем функцию от значений
                # мета-информация никак не меняется
                ts = TimeSeries(values=fun(self.values, other.values), 
                                start_period=self.start_period, end_period=self.end_period,
                                diff_level=self.diff_level)
                return ts
            
            # скаляры применяются к значениям безусловно, при этом мета-информация никак не меняется
            elif isinstance(other, (int, float)):
                ts = TimeSeries(fun(self.values, other),
                                start_period=self.start_period, end_period=self.end_period,
                                diff_level=self.diff_level)
                return ts
            else:
                raise ValueError('argument has incompatible type')
    
    # Все необходимые операции выражаются через apply
    def __add__(self, other):
        return self.apply(lambda x, y: x + y, other)
    
    def __sub__(self, other):
        return self.apply(lambda x, y: x - y, other)
    
    def __truediv__(self, other):
        return self.apply(lambda x, y: x / y, other)
    
    def cumprod(self):
        return self.apply(lambda x: x.cumprod())
    
    def __repr__(self):
        return 'TimeSeries(start_period={}, end_period={}, diff_level={}, values={}'.format(
            self.start_period, self.end_period, self.diff_level, self.values
        )

Имея такую надстройку, следующая попытка ожидаемо привдёт к ошибке из-за несовместимости периодов (`start periods are incompatible`):

In [17]:
x = TimeSeries(values=np.array([4, 2]), 
               start_period=pd.Period('2015-1', freq='M'), 
               end_period=pd.Period('2015-2', freq='M'), 
               diff_level=1)

y = TimeSeries(values=np.array([1, 2]), 
               start_period=pd.Period('2015-2', freq='M'), 
               end_period=pd.Period('2015-3', freq='M'), 
               diff_level=1)

x / y

ValueError: start periods are incompatible

Теперь посчитаем реальную доходность для индекса S&P с помощью `TimeSeries`:

In [18]:
snp_ror_ts = TimeSeries(
    # Для простоты, выше не были введены операции `diff` и адресации массива, что позволило сделать так:
    # snp_ts = TimeSeries(...)
    # snp_ror_ts = snp_ts.diff() / snp_ts[:-1]
    # Это можно проделать в качестве упражнения
    values=np.diff(snp_values) / snp_values[:-1], 
    start_period=pd.Period('2015-2', freq='M'),
    end_period=pd.Period('2016-3', freq='M'),
    diff_level=1,
)

infl_usd_ts = TimeSeries(
    values=infl_usd_values[1:],
    start_period=pd.Period('2015-2', freq='M'),
    end_period=pd.Period('2016-3', freq='M'),
    diff_level=1,
)

snp_ror_real_ts = (snp_ror_ts + 1.) / (infl_usd_ts + 1.) - 1.
snp_ror_real_ts

TimeSeries(start_period=2015-02, end_period=2016-03, diff_level=1, values=[ 0.05163734 -0.02156758  0.00778508  0.0077198  -0.02371087  0.02252073
 -0.05961855 -0.02391243  0.08554818  0.00577832 -0.01380723 -0.05135471
 -0.00164759  0.06266874]

In [19]:
# check:
snp_asset.rate_of_return(real=True).values - snp_ror_real_ts.values

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

Возможны случаи, когда временной ряд после применения операции преобразуется в одно число. Например, `CAGR`. `CAGR` не не вписывается в понятие временного ряда, `TimeSeries`. Поэтому лучше ввести дополнительный класс `TimeValue`, с идентичной метаинформацией, а далее при необходимости расширить список типов аргументов для второго параметра в функции `TimeSeries.apply`.


В следующей статье мы рассмотрим пример вычисления портфеля из 2х активов.